# Predicting Potency for human GCGR/GLP-1 Receptors Using a Pre-Trained Ensemble

This notebook offers a trutorial on how to use a pre-trained enesamble of muli-task neural network models to predict potencies.

## Imports

In [ ]:
import time
from pathlib import Path

import numpy as np
import pandas as pd
from Bio import SeqIO
from keras.models import load_model
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import warnings

from keras.models import load_model

from peptide_models.aminoacids import AMINOACIDS
from peptide_models.peptide import Peptide
from peptide_models.utils_models import get_models
from peptide_models.utils_data import pep2fasta
from peptide_models.predict_main import main
from peptide_models.utils_plotting import get_plot

warnings.filterwarnings("ignore")

## Load pre-trained models

In [ ]:
path_to_models = Path('../models')
print("Loading models ...")
ensemble = get_models(path_models=path_to_models)

## Examples

## 1. Predict potencies against human GCGR/GLP-1R receptors for dummy data using a pre-trained ensemble

### Generate a set of random peptide sequences

In [ ]:
def get_random_seqs(n_samples=100, length_seq=30):
    """
    Get random sequences for testing 
    :param n_samples: 
    :return: 
    """
    samples = []
    for i in range(n_samples):
        samples.append("".join(np.random.choice([a.letter for a in AMINOACIDS],
                                        size=length_seq,
                                        replace=True)))
    return samples

In [ ]:
seqs = get_random_seqs()

In [ ]:
# Instantiate peptide objects 
random_peptides = []
for idx in range(len(seqs)):
    peptide = Peptide(alias='test_set',
                      ec_50A=None,
                      ec_50B=None,
                      name=idx,
                      sequence=seqs[idx],
                      c_term=True)
    random_peptides.append(peptide)

### Save FASTA file with dummy peptide sequences 

In [ ]:
if not Path('../data/FASTA_files/dummy_data').exists():
    Path('../data/FASTA_files/dummy_data').mkdir(parents=True)
    print('Created path to store data.')
    
    pep2fasta(peptides_list=random_peptides,
              output_path=Path('../data/FASTA_files/dummy_data'),
              dataset_name='random_peptides')

In [ ]:
for pep in random_peptides:
    pep.predict_potency(infers=ensemble)

for i, p in enumerate(random_peptides):
    print(f'Random peptide:{i}, predicted potency at GCGCR:{p.ec_50A.round(2)}', 
          f'predicted potenct at GLP-1R:{p.ec_50B.round(2)}')

## 2. Predict potencies against human GCGR/GLP-1R receptors for training set sequences using a pre-trained ensemble

### Load training data 

In [ ]:
data_path = Path('../data/training_data.xlsx')

In [ ]:
dataset = pd.read_excel(str(data_path), index_col=0, header=0, skiprows=0, sheet_name='dataset')
msa = pd.read_excel(str(data_path), index_col=0, header=0, skiprows=0, sheet_name='alignment')

In [ ]:
# Get training peptides
training_peptides = []
for idx in range(len(dataset)):
    pep_record = dataset.iloc[idx]
    peptide = Peptide(alias=pep_record.alias,
                      ec_50A=pep_record.EC50_LOG_T1,
                      ec_50B=pep_record.EC50_LOG_T2,
                      name=idx,
                      sequence=msa.iloc[idx].sequence,
                      c_term=True)
    training_peptides.append(peptide)

In [ ]:
# True values of potency measured for training peptides
y_true = np.asarray([[p.ec_50A for p in training_peptides], [p.ec_50B for p in training_peptides]])

In [ ]:
# Remove the true potencies
training_peptides = []
for idx in range(len(dataset)):
    pep_record = dataset.iloc[idx]
    peptide = Peptide(alias=pep_record.alias,
                      ec_50A=None,
                      ec_50B=None,
                      name=idx,
                      sequence=msa.iloc[idx].sequence,
                      c_term=True)
    training_peptides.append(peptide)

In [ ]:
# Predict the potencies for training set sequences 
for pep in training_peptides:
    pep.predict_potency(infers=ensemble)

for i, p in enumerate(training_peptides):
    print(f'Training peptide:{i}, predicted potency at GCGCR:{p.ec_50A.round(2)}', 
          f'predicted potenct at GLP-1R:{p.ec_50B.round(2)}')

In [ ]:
# Plot predicted potencies for training peptides
y_predicted = np.asarray([[p.ec_50A for p in training_peptides],[p.ec_50B for p in training_peptides]])

In [ ]:
if not Path('../results/predictions/').exists():
    Path('../results/predictions/').mkdir(parents=True)
    print('Created path to store data.')
get_plot(y_pred=y_predicted, y_test=y_true, path_to_figs='../results/predictions/', name='training_data')

# Predicting Potency for human GCGR/GLP-1 Receptors Using a Pre-Trained Ensemble

Please specify the path to the folder with FASTA files below.

### Requirements: 

- Sequence Data Format: Please ensure that the sequence data intended for prediction is stored in __FASTA__ format.

- Sequence Length: Each sequence must consists of exactly __30__ amino acids (characters).

### In the example below, we predict for 288 natural __glucagon and GLP-1 orthologous sequences from various organisms__ collected from the [__NCBI database__](https://www.ncbi.nlm.nih.gov).

Please ensure that the __out_path__ is empty before running.

In [ ]:
path_to_models = Path('../models')
#Path to yout data
data_path = Path('../data/FASTA_files/NCBI_data')
#Path to save your prediction results
out_path = Path('../results/predictions/NCBI_data')

In [ ]:
main(models_path=path_to_models,
     output_path=out_path,
     data_folder=data_path)

### The software output is an Excel spreadsheet with the predictions.

In [ ]:
#Load the saved file from the output folder
pd.read_excel('../results/predictions/NCBI_data/GLP1.xlsx', index_col=0, header=0, skiprows=0)